In [ ]:
'''Trains a memory network on the bAbI dataset.
References:
- Jason Weston, Antoine Bordes, Sumit Chopra, Tomas Mikolov, Alexander M. Rush,
  "Towards AI-Complete Question Answering: A Set of Prerequisite Toy Tasks",
  http://arxiv.org/abs/1502.05698
- Sainbayar Sukhbaatar, Arthur Szlam, Jason Weston, Rob Fergus,
  "End-To-End Memory Networks",
  http://arxiv.org/abs/1503.08895
'''
from __future__ import print_function

from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate
from keras.layers import LSTM
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from functools import reduce
import matplotlib.pyplot as plt

import os
import numpy as np
import re

def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]

def get_stories(filename):
    '''Given a file name, read the file,
    retrieve the stories,
    and then convert the sentences into a single story.
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    Parse stories provided in the bAbi tasks format
    If only_supporting is true, only the sentences
    that support the answer are kept.
    '''
    f = open(filename, "r")
    lines = f.readlines()
    data = []
    story = []
    for line in lines:
        if '\t' in line:
            question, answer, ind_s, ind_e = line.split('\t')
            ind_s = int(ind_s)
            ind_e = int(ind_e)
            question = question.split()
            data.append((story, question, answer, ind_s, ind_e))
        else:
            nid, context = line.split(' ', 1)
            story = context.split()
    f.close()
    return data

def vectorize_stories(data, word_idx, story_maxlen, query_maxlen, vocab_size):
    X = []
    Xq = []
    Y = []
    for story, query, answer, ind_s, ind_e in data:
        x = [word_idx[w] for w in story]
        xq = [word_idx[w] for w in query]
        #a = [word_idx[w] for w in answer]
        # let's not forget that index 0 is reserved
        y = np.zeros(story_maxlen + 1)
        y[ind_s + 1] = 1
        #[ind_s:ind_e+1] = 1
        X.append(x)
        Xq.append(xq)
        Y.append(y)
    return (pad_sequences(X, maxlen=story_maxlen),
            pad_sequences(Xq, maxlen=query_maxlen), np.array(Y))


In [ ]:
challenges = {
    # Squad sample for test
    'squad_sample': './sample_{}.txt',
    # Squad
    'squad': './{}.txt',
}
challenge_type = 'squad_sample'

challenge = challenges[challenge_type]

print('Extracting stories for the challenge:', challenge_type)

train_stories= get_stories(challenge.format('train'))
test_stories = get_stories(challenge.format('test'))

vocab = set()
for story, question, answer, ind_s, ind_e in train_stories + test_stories:
    vocab |= set(story + question)
vocab = sorted(vocab)

# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
story_maxlen = max(map(len, (x for x, _, _,_,_ in train_stories + test_stories)))
query_maxlen = max(map(len, (x for _, x, _,_,_ in train_stories + test_stories)))

print('-')
print('Vocab size:', vocab_size, 'unique words')
print('Story max length:', story_maxlen, 'words')
print('Query max length:', query_maxlen, 'words')

print('Number of training stories:', len(train_stories))
print('Number of test stories:', len(test_stories))
print('-')
print('Here\'s what a "story" tuple looks like (input, query, answer):')
print(train_stories[0])
print('-')
print('Vectorizing the word sequences...')

word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
inputs_train, queries_train, answers_train = vectorize_stories(train_stories,
                                                               word_idx,
                                                               story_maxlen,
                                                               query_maxlen,
                                                               vocab_size)
inputs_test, queries_test, answers_test = vectorize_stories(test_stories,
                                                            word_idx,
                                                            story_maxlen,
                                                            query_maxlen,
                                                            vocab_size)

print('-')
print('inputs: integer tensor of shape (samples, max_length)')
print('inputs_train shape:', inputs_train.shape)
print('inputs_test shape:', inputs_test.shape)
print('-')
print('queries: integer tensor of shape (samples, max_length)')
print('queries_train shape:', queries_train.shape)
print('queries_test shape:', queries_test.shape)
print('-')
print('answers: binary (1 or 0) tensor of shape (samples, vocab_size)')
print('answers_train shape:', answers_train.shape)
print('answers_test shape:', answers_test.shape)
print('-')
print('Compiling...')

In [ ]:
# placeholders
input_sequence = Input((story_maxlen,))
question = Input((query_maxlen,))

# encoders
# embed the input sequence into a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,
                              output_dim=64))
input_encoder_m.add(Dropout(0.3))
# output: (samples, story_maxlen, embedding_dim)

# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,
                              output_dim=query_maxlen))
input_encoder_c.add(Dropout(0.3))
# output: (samples, story_maxlen, query_maxlen)

# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=query_maxlen))
question_encoder.add(Dropout(0.3))
# output: (samples, query_maxlen, embedding_dim)

# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

# compute a 'match' between the first input vector sequence
# and the question vector sequence
# shape: `(samples, story_maxlen, query_maxlen)`
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

# concatenate the match matrix with the question vector sequence
answer = concatenate([response, question_encoded])

# the original paper uses a matrix multiplication for this reduction step.
# we choose to use a RNN instead.
answer = LSTM(32)(answer)  # (samples, 32)

# one regularization layer -- more would probably be needed.
answer = Dropout(0.3)(answer)
answer = Dense(story_maxlen+1)(answer)  # (samples, story_sentence_maxlen)
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

# build the final model
model = Model([input_sequence, question], answer)
model.summary()
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
print(inputs_train.shape)
print(queries_train.shape)
print(answers_train.shape)

In [ ]:
# save best model
MODEL_DIR = "Z:/jj_git/elice/JARDIS/MEMN2N_BASE_MODEL/checkpoint_"+challenge_type
checkpoint = ModelCheckpoint(filepath = os.path.join(MODEL_DIR, "model-{epoch:02d}.h5"),save_best_only=True)

# train
history = model.fit([inputs_train, queries_train], answers_train,
          batch_size=32,
          epochs=3,
          verbose=2, validation_data=([inputs_test, queries_test], answers_test),callbacks = [checkpoint])


score = model.evaluate([inputs_test, queries_test], answers_test, batch_size=32, verbose = 1)
print(model.metrics_names)
print("Test loss: ", score[0])
print("Test accuracy: ", score[1])

pred = model.predict([inputs_test, queries_test])
pred_train = model.predict([inputs_train, queries_train])

In [ ]:

# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.figure()
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'],loc = 'upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'],loc = 'upper left')
plt.show()

In [ ]:

index = 1
question = queries_train[index]
sentence_index = int(train_stories[index][2])
story_index = inputs_train[index]
print(sentence_index)

story = train_stories[index][0]
question = train_stories[index][1]
answer = train_stories_ori[index][0][sentence_index]
prediction = train_stories_ori[index][0][np.argmax(pred_train[index])-1]
print('Story:')
print(' '.join(word for word in story))
print('\nQuestion:')
print(' '.join(word for word in question))
print('\nPrediction (sentence):')
print(' '.join(word for word in prediction))
print('\nCorrect:')
print(np.argmax(pred_train[index]) == sentence_index)